# Kruskal-Wallis-Test – UX-Gruppen vergleichen ohne Verteilungsannahme

Autor: Luca Honegger, [lh@sinnhaft.ch](mailto:lh@sinnhaft.ch)
Quelle: Dies ist eine Python-Portierung der UXtoolbox in R von Mohsen Rafiei, [https://github.com/mohsen-rafiei/UXtoolbox](https://github.com/mohsen-rafiei/UXtoolbox)


## 🧠 Was macht diese Funktion?

Die Funktion `kruskal_wallis_test()` hilft dir dabei, **mehrere Gruppen in deiner UX-Studie zu vergleichen**, **ohne** dass du Normalverteilung oder Varianzgleichheit voraussetzen musst.

Im Gegensatz zu einer klassischen ANOVA oder einem bayesschen Modell prüft diese Funktion rein **ranganalytisch**, ob sich die Gruppen **signifikant unterscheiden** – also:

* ob z. B. Design A, B und C **unterschiedliche Verteilungen** aufweisen,
* ob **zumindest eine Gruppe** sich statistisch messbar abhebt.

Zusätzlich berechnet sie:

* die **Effektstärke (η²)** als Mass für die Grösse des Unterschieds,
* sowie **Post-hoc-Vergleiche zwischen allen Gruppenpaaren**, inklusive Bonferroni-Korrektur.


### 🔍 Was wird berechnet?

Die Funktion umfasst drei Schritte:

1. **Kruskal-Wallis-Test**
   → prüft auf allgemeine Unterschiede zwischen 3+ Gruppen

2. **Effektstärke η²**
   → zeigt, **wie gross** der Unterschied ist (nicht nur ob signifikant)

3. **Paarweise Vergleiche** (Mann-Whitney U mit Bonferroni-Korrektur)
   → zeigt, **welche Gruppen sich voneinander unterscheiden**

Das Verfahren ist **nicht-parametrisch** – ideal, wenn du dir **nicht sicher bist, ob die Daten normalverteilt sind**.

### 🧪 Beispiel aus der Praxis

Du hast z. B. folgende Daten aus einem UX-Test:

| UX\_Score | Design |
| --------- | ------ |
| 72        | A      |
| 76        | B      |
| 85        | C      |
| …         | …      |

Dann kannst du mit folgendem Aufruf testen, ob sich die Designs in ihren Scores unterscheiden:

```python
results = kruskal_wallis_test(data, group_col="Design", value_col="UX_Score")
```

Und dir die Ergebnisse ansehen:

```python
print(results["Kruskal_Wallis_Test"])
print(results["Effect_Size"])
print(results["Pairwise_Comparisons"])
```


### 🎯 Warum ist das nützlich?

Wenn du kleine UX-Samples hast (z. B. n = 5 pro Gruppe), oder wenn du **keine Normalverteilung annehmen möchtest**, ist `kruskal_wallis_test()` eine **robuste Alternative zur ANOVA**.

Sie zeigt dir:

* ob sich Gruppen **statistisch unterscheiden**,
* wie gross dieser Unterschied **praktisch relevant** ist,
* und **zwischen welchen Gruppen** die Unterschiede bestehen.

Das hilft dir z. B. in A/B/C-Tests fundierte Entscheidungen zu treffen – auch ohne ausgefeilte Modellannahmen.


### 📝 Hinweise

**✅ Wann ist die Funktion geeignet?**

* Wenn deine Zielvariable **ordinal oder intervallskaliert** ist
* Wenn du **3 oder mehr Gruppen** vergleichen willst
* Wenn du **robust gegen Ausreisser und Verteilungsprobleme** vorgehen möchtest

**❌ Wann nicht?**

* Wenn du mit **binären Zielvariablen** arbeitest
* Wenn du **exakte Wahrscheinlichkeiten oder Unsicherheiten** brauchst (→ Bayes)
* Wenn du **nur 2 Gruppen** vergleichen willst (→ `mannwhitneyu()` direkt)

**🟢 Was dann stattdessen?**

* Für 2 Gruppen: → Mann-Whitney-U-Test (direkt)
* Für Wahrscheinlichkeiten: → `bayesian_anova()` oder `bayesian_logistic_regression()`
* Für normalverteilte metrische Daten: → klassische ANOVA

In [62]:
import pandas as pd
import numpy as np
from scipy.stats import kruskal
from itertools import combinations
from statsmodels.stats.multitest import multipletests

def kruskal_wallis_test(data, group_col, value_col):
    """
    Führt einen Kruskal-Wallis-Test durch für ordinal- oder intervallskalierte UX-Daten.

    Die Funktion testet, ob es signifikante Unterschiede zwischen drei oder mehr unabhängigen Gruppen
    gibt (z. B. verschiedene Design-Varianten oder Nutzergruppen), ohne die Annahme der Normalverteilung.
    Zusätzlich wird die Effektstärke (η²) berechnet sowie paarweise Post-hoc-Vergleiche (Mann-Whitney U)
    mit Bonferroni-Korrektur durchgeführt.

    Parameter:
    - data: Pandas DataFrame mit den UX-Daten
    - group_col: Spaltenname der Gruppierungsvariable (z. B. "Group")
    - value_col: Spaltenname der numerischen Zielgrösse (z. B. "Score")

    Rückgabe:
    - Dictionary mit:
        - 'Kruskal_Wallis_Test': Teststatistik und p-Wert des Kruskal-Wallis-Tests
        - 'Effect_Size': η² als Mass für die Effektstärke
        - 'Pairwise_Comparisons': DataFrame mit Post-hoc-Vergleichen und adjustierten p-Werten
    """
    
    # 1. Gruppen extrahieren
    groups = data[group_col].unique()
    group_data = [data[data[group_col] == g][value_col] for g in groups]

    # 2. Kruskal-Wallis-Test
    stat, p_value = kruskal(*group_data)
    k_result = {'H-statistic': stat, 'p-value': p_value}

    # 3. Effektstärke η² berechnen
    n_total = len(data)
    eta_squared = (stat - len(groups) + 1) / (n_total - 1)
    eff_result = {'eta_squared': eta_squared}

    # 4. Paarweise Vergleiche (Dunn-ähnlich mit Mann-Whitney U)
    comparisons = list(combinations(groups, 2))
    pvals = []

    for g1, g2 in comparisons:
        x = data[data[group_col] == g1][value_col]
        y = data[data[group_col] == g2][value_col]
        from scipy.stats import mannwhitneyu
        _, p = mannwhitneyu(x, y, alternative='two-sided')
        pvals.append(p)

    # Bonferroni-Korrektur
    reject, pvals_corrected, _, _ = multipletests(pvals, method='bonferroni')

    # Ergebnisse zusammenfassen
    posthoc_results = pd.DataFrame({
        'Group1': [g1 for g1, g2 in comparisons],
        'Group2': [g2 for g1, g2 in comparisons],
        'p-value_raw': pvals,
        'p-value_adj': pvals_corrected,
        'significant': reject
    })

    return {
        'Kruskal_Wallis_Test': k_result,
        'Effect_Size': eff_result,
        'Pairwise_Comparisons': posthoc_results
    }


## 🚀 Anwendungsbeispiel

In [65]:
import pandas as pd

# Beispieldaten: Drei verschiedene Designs, System Usability Scale (SUS)-Scores
data = {
    'Design': ['A', 'A', 'A', 'A', 'A',
               'B', 'B', 'B', 'B', 'B',
               'C', 'C', 'C', 'C', 'C'],
    
    'UX_Score': [72, 75, 78, 74, 76,   # Gruppe A: mittlere Zufriedenheit
                 73, 76, 79, 75, 77,   # Gruppe B: fast gleich wie A
                 83, 85, 87, 84, 86]   # Gruppe C: sichtbar höher
}

# DataFrame erstellen
df = pd.DataFrame(data)

In [67]:
# Durchführung des Kruskal-Wallis-Tests
ergebnisse = kruskal_wallis_test(df, group_col='Design', value_col='UX_Score')

# Ausgabe der Resultate
print("Kruskal-Wallis-Test:")
print(ergebnisse['Kruskal_Wallis_Test'])

print("\nEffektstärke η²:")
print(ergebnisse['Effect_Size'])

print("\nPost-hoc-Vergleiche mit Bonferroni-Korrektur:")
print(ergebnisse['Pairwise_Comparisons'])


Kruskal-Wallis-Test:
{'H-statistic': np.float64(9.654480286738355), 'p-value': np.float64(0.008008593363531068)}

Effektstärke η²:
{'eta_squared': np.float64(0.5467485919098826)}

Post-hoc-Vergleiche mit Bonferroni-Korrektur:
  Group1 Group2  p-value_raw  p-value_adj  significant
0      A      B     0.528359      1.00000        False
1      A      C     0.007937      0.02381         True
2      B      C     0.007937      0.02381         True


### 📖 Erklärung und Interpretation

#### Überblick: Was zeigt der Test?

Es wurde ein **Kruskal-Wallis-Test** durchgeführt, um zu prüfen, ob sich die UX-Scores (z. B. aus einem SUS-Fragebogen) **zwischen den Gruppen A, B und C signifikant unterscheiden**.
Da der Test nicht-parametrisch ist, eignet er sich für ordinalskalierte Daten oder bei Nicht-Normalverteilung.


#### Testergebnis – Interpretation

| Test                | Ergebnis |
| ------------------- | -------- |
| **H-Statistik**     | 9.65     |
| **p-Wert**          | 0.0080   |
| **Effektstärke η²** | 0.547    |

* ✅ Der **p-Wert ist < 0.05** → Es gibt **signifikante Unterschiede** zwischen mindestens zwei Gruppen.
* ✅ Die **Effektstärke η² = 0.55** zeigt einen **starken Unterschied** zwischen den Gruppen (Faustregel: > 0.14 = grosser Effekt).



#### Paarweise Post-hoc-Vergleiche (Bonferroni-korrigiert)

| Gruppenvergleich | p-Wert (adj) | Signifikant? | Interpretation                               |
| ---------------- | ------------ | ------------ | -------------------------------------------- |
| A vs. B          | 1.000        | ❌ nein       | Keine signifikanten Unterschiede             |
| A vs. C          | 0.0238       | ✅ ja         | Gruppe C hat signifikant höhere Scores als A |
| B vs. C          | 0.0238       | ✅ ja         | Gruppe C hat signifikant höhere Scores als B |

* Gruppe C ist **signifikant besser** als die anderen beiden.
* Zwischen A und B gibt es **keinen Unterschied**.



#### Business-Interpretation

#### Möglicher Kontext:

Ein UX-Team testet drei Design-Varianten (A, B, C) einer Anwendung. Nutzende bewerten das Design mit dem SUS (Scorebereich 0–100).

#### Ergebnisse & Empfehlungen:

| Erkenntnis                          | Auswirkung                                                                                     |
| ----------------------------------- | ---------------------------------------------------------------------------------------------- |
| ✅ Gruppe C hebt sich signifikant ab | Design C hat **nachweislich höhere UX-Werte** – sollte bevorzugt oder weiter optimiert werden. |
| ❌ A vs. B kein Unterschied          | Design A und B sind **gleichwertig** – Redundanzen können geprüft oder reduziert werden.       |
| 📊 Grosser Effekt gesamt (η² = 0.55) | Der Unterschied ist nicht nur statistisch, sondern **auch praktisch relevant**.                |


#### 📌 Fazit

| Massnahme                                           | Begründung                                        |
| -------------------------------------------------- | ------------------------------------------------- |
| ✅ **Design C priorisieren**                        | Signifikant und praktisch besser                  |
| 🔍 A und B zusammenführen oder A/B-Test einstellen | Kein Unterschied → Ressourcen sparen              |
| 🧪 Qualitative Forschung zu Design C durchführen   | Verstehen, **was genau** zu der besseren UX führt |


---

➡️ **Tipp**: Für eine Bayes’sche Alternative zum Kruskal-Wallis-Test verwende eine Bayes’sche ANOVA (siehe `bayesian_anova.ipynb`).